In [35]:
# importing the lib
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import numpy
import time
import sys
from selenium.webdriver.chrome.service import service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import re
import numpy as np

In [46]:
def kayak(doj,souce_airport,destination_airport):
    url="https://www.ca.kayak.com/flights/" +str(source_airport) +"-" +str(destination_airport)+ "/"+str(doj)+ "?sort=bestflight_a"
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.maximize_window()
    driver.get(url)
    time.sleep(15)
    def click_more():
    
        for val in range(0,50):
            try:
                driver.find_element_by_xpath("//a[@class='moreButton']").click()
                time.sleep(2)
                val = val +1

            except:
                pass
    click_more()
    
    # depart-time base-time
    departure_time=[]
    departure_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='depart-time base-time']")

    arrival_time=[]
    arrival_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='arrival-time base-time']")

    duration =[]
    duration  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section duration allow-multi-modal-icons']//div[@class='top']")


    no_stop = []
    no_stop  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section stops']//div[@class='top']")

    price =[]
    price = driver.find_elements_by_xpath("//div[@class = 'booking']")

    airlines = []
    airlines = driver.find_elements_by_xpath("//span[@class = 'codeshares-airline-names']")

    # departure_time = driver.find_elements_by_xpath("//div[@class='itinerary-depart-time depart-time']")

    
    
    
    price_list=[]
    stops=[]
    for i in range(len(departure_time)):
        departure_time[i]= departure_time[i].get_attribute('innerHTML')
        arrival_time[i] = arrival_time[i].get_attribute('innerHTML')
        duration[i] = duration[i].get_attribute('innerHTML')
        no_stop[i] = no_stop[i].get_attribute('innerHTML')
        l = re.findall("(direct|1|2|3|4|5)",no_stop[i])[0]
        stops.append(l)
        price[i] = price[i].get_attribute('innerHTML')

#         airlines[i] = airlines[i].get_attribute('innerHTML')
        b = re.findall("nbsp\;((\d\d\d)|(\d.\d\d\d)|(\d\d.\d\d\d))",price[i])[0][0]
        price_list.append(b)
    
    for k in range(len(airlines)):
        airlines[k] = airlines[k].get_attribute('innerHTML')
        
    # Creating data frames
    departure = len(departure_time)*[str(source_airport)]
    arrival = len(departure_time)*[str(destination_airport)]
    # a = pd.DataFrame(departure_time,arrival_time,price_list)
    doj1 = len(departure_time)*[doj]
    df  = pd.DataFrame(doj1,columns=["Date of journey"])
    df["Source"] = departure    
    df["Destination"] = arrival
    df["Departure_time"] = departure_time
    df["Arrival_time"] = arrival_time
    df["Duration"] = duration
    df["Stops"] = stops
    df["CAD $"]= price_list
    df["Airlines"] = airlines
#     df["Stops"]=df["Stops"].astype(str).str.replace("\n","")
    df["CAD $"]=df["CAD $"].astype(str).str.replace(",","")
    df["CAD $"] = pd.to_numeric(df['CAD $'])
#     df["Stops"]=df["Stops"].astype(str).str.replace("stops","")
#     df["Stops"]=df["Stops"].astype(str).str.replace("stop","")
    df["Duration"]=df["Duration"].astype(str).str.replace("\n","")
    df = df.drop(df[(df.Stops =='3') | (df.Airlines == 'Multiple Airlines')].index)
    df= df.drop_duplicates()
    df = df.sort_values(by='CAD $')
    df = df.reset_index()
    df.index = df.index +1
    df.pop("index")
    driver.close()
    return df
    
    

In [49]:
source = ["YYZ","YYC","YVR","YQB","YHZ"]
destination = ["DEL","BOM","BLR","AMD","IXC"]
# source = ["YVR"]
# destination = ["BOM"]

doj='2022-05-30'
df1 = pd.DataFrame(columns=['Date of journey', 'Source', 'Destination', 'Departure_time',
                            'Arrival_time', 'Duration', 'Stops', 'CAD $', 'Airlines'])
for val in source:
    for values in destination:
        source_airport = val
        print(source_airport)
        
        
        destination_airport = values
        print(destination_airport)
        doj ='2022-05-30'
        df = kayak(doj,source_airport,destination_airport)
        print("Departure length",len(departure_time))
#         print(df1.describe())
        df1=pd.concat([df1,df])
        



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Haet\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


YYZ
DEL


<ipython-input-46-bb81a9d55b4f>:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
<ipython-input-46-bb81a9d55b4f>:11: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//a[@class='moreButton']").click()
<ipython-input-46-bb81a9d55b4f>:21: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  departure_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='depart-time base-time']")
<ipython-input-46-bb81a9d55b4f>:24: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  arrival_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='arrival-time base-time']")
<ipython-input-46-bb81a9d55b4f>:27: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elemen

Departure length 117
YYZ
BOM




====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Haet\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


...

Departure length 117
YHZ
BLR




====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome


Departure length 117
YHZ
AMD


Driver [C:\Users\Haet\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Haet\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache


Departure length 117
YHZ
IXC
Departure length 117


In [51]:
df1.describe()

,Date of journey,Source,Destination,Departure_time,Arrival_time,Duration,Stops,CAD $,Airlines
count,767,767,767,767,767,767,767,767,767
unique,1,5,5,120,101,282,3,409,82
top,2022-05-30,YVR,BOM,4:15 p.m.,1:00 a.m.,27h 30m,2,1816,Air Canada
freq,767,379,340,44,62,10,630,14,137


In [61]:
df2=df1.copy()
df2[ (df2.Destination=="AMD")]

,Date of journey,Source,Destination,Departure_time,Arrival_time,Duration,Stops,CAD $,Airlines
1,2022-05-30,YYZ,AMD,9:40 p.m.,4:25 a.m.,21h 15m,1,952,Etihad Airways
2,2022-05-30,YYZ,AMD,12:15 p.m.,8:35 p.m.,22h 50m,1,1115,Air India
3,2022-05-30,YYZ,AMD,6:35 p.m.,4:25 a.m.,24h 20m,2,1379,"Air Canada, Etihad Airways"
4,2022-05-30,YYZ,AMD,6:35 p.m.,3:05 a.m.,23h 00m,2,1755,"Air Canada, Emirates"
5,2022-05-30,YYZ,AMD,5:59 p.m.,2:15 a.m.,22h 46m,2,1784,"American Airlines, Qatar Airways"
6,2022-05-30,YYZ,AMD,5:59 p.m.,2:15 a.m.,22h 46m,2,1787,Qatar Airways
7,2022-05-30,YYZ,AMD,5:50 p.m.,2:15 a.m.,22h 55m,2,2431,"WestJet, Qatar Airways"
8,2022-05-30,YYZ,AMD,7:00 p.m.,2:15 a.m.,21h 45m,2,3207,"Air Canada, Qatar Airways"
9,2022-05-30,YYZ,AMD,6:35 p.m.,2:15 a.m.,22h 10m,2,3980,"British Airways, Qatar Airways"
10,2022-05-30,YYZ,AMD,6:20 p.m.,4:25 a.m.,24h 35m,2,4766,"Air Canada, Etihad Airways"


In [26]:
source_airport="YYZ"
destination_airport="BLR"
doj='2022-05-30'
url="https://www.ca.kayak.com/flights/" +str(source_airport) +"-" +str(destination_airport)+ "/"+str(doj)+ "?sort=bestflight_a"
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
driver.get(url)
time.sleep(15)
def click_more():

    for val in range(0,50):
        try:
            driver.find_element_by_xpath("//a[@class='moreButton']").click()
            time.sleep(4)
            val = val +1

        except:
            pass
click_more()



====== WebDriver manager ======
Current google-chrome version is 99.0.4844
Get LATEST chromedriver version for 99.0.4844 google-chrome
Driver [C:\Users\Haet\.wdm\drivers\chromedriver\win32\99.0.4844.51\chromedriver.exe] found in cache
<ipython-input-26-dae93289739f>:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())
<ipython-input-26-dae93289739f>:13: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath("//a[@class='moreButton']").click()


In [27]:
departure_time=[]
departure_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='depart-time base-time']")

arrival_time=[]
arrival_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='arrival-time base-time']")

duration =[]
duration  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section duration allow-multi-modal-icons']//div[@class='top']")

no_stop = []
no_stop  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section stops']//div[@class='top']")

price =[]
price = driver.find_elements_by_xpath("//div[@class = 'booking']")

airlines = []
airlines = driver.find_elements_by_xpath("//span[@class = 'codeshares-airline-names']")

<ipython-input-27-b14ccfb4e37d>:2: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  departure_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='depart-time base-time']")
<ipython-input-27-b14ccfb4e37d>:5: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  arrival_time= driver.find_elements_by_xpath("//ol[@class='flights']//span[@class='arrival-time base-time']")
<ipython-input-27-b14ccfb4e37d>:8: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  duration  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section duration allow-multi-modal-icons']//div[@class='top']")
<ipython-input-27-b14ccfb4e37d>:11: DeprecationWarning: find_elements_by_* commands are deprecated. Please use find_elements() instead
  no_stop  = driver.find_elements_by_xpath("//ol[@class='flights']//div[@class='section stops']

In [29]:
len(no_stop)

117

In [30]:
stops=[]

In [25]:
type(no_stop[1])

selenium.webdriver.remote.webelement.WebElement

In [31]:
for i in range(len(departure_time)):
    no_stop[i] = no_stop[i].get_attribute('innerHTML')
    l = re.findall("(direct|1|2|3|4|5)",no_stop[i])[0]
    stops.append(l)